In [ ]:
# pip uninstall spark-nlp

In [ ]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

import warnings
warnings.filterwarnings('ignore')

import os 
os.environ['PYSPARK_PYTHON'] = "/root/anaconda3/bin/python"     # 设置了运行 PySpark 的 Python 解释器的环境变量



In [ ]:
from kafka import KafkaConsumer
import json 

broker = 'localhost:64050'
consumer = KafkaConsumer('q1_3', 
        bootstrap_servers=broker, 
        value_deserializer=lambda m: json.loads(m.decode('utf-8')), 
        consumer_timeout_ms=100000,
        auto_offset_reset='earliest', enable_auto_commit=False
    )

In [ ]:
# spark = SparkSession.builder \
#       .appName("Spark NLP") \
#       .master("local[*]") \
#       .config("spark.driver.memory", "16G") \
#       .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
#       .config("spark.kryoserializer.buffer.max", "2000M") \
#       .config("spark.driver.maxResultSize", "0") \
#       .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
#       .config("spark.jars", "/shareddata/lab09/kafka-clients-3.5.0.jar,/shareddata/lab09/spark-sql-kafka-0-10_2.12-3.5.0.jar,/shareddata/lab09/spark-token-provider-kafka-0-10_2.12-3.5.0.jar,/shareddata/lab09/commons-pool2-2.12.0.jar") \
#       .getOrCreate()

In [ ]:
spark1 = sparknlp.start()

In [ ]:
MODEL_NAME = "best_model"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

T5 = T5Transformer.load(f"{MODEL_NAME}_spark_nlp") \
  .setInputCols(["document"]) \
  .setOutputCol("summary")


pipeline = Pipeline().setStages([document_assembler, T5])


In [ ]:
for message in consumer:
    input = message.value["input"]
    output = message.value["output"]
    # print(input)
    data_df = spark1.createDataFrame([[input]]).toDF("text")
    results = pipeline.fit(data_df).transform(data_df)
    results.select("summary.result").show(truncate=False)
